In [33]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [34]:
#----------資料讀入並進行篩選----------
import pandas as pd
CrudeOilETF= pd.read_csv('Crude Oil ETF List (22).csv') #讀檔
fliter = (CrudeOilETF["Inception"] < '2015-12-31') #Inception條件判斷進行篩選
newCrudeOilETF = CrudeOilETF[fliter] #Crude Oil ETF List篩選結果

GoldETF= pd.read_csv('Gold ETF List (17).csv') #讀檔
fliter = (GoldETF["Inception"] < '2015-12-31') #Inception條件判斷進行篩選
newGoldETF = GoldETF[fliter] #Gold ETF List篩選結果

#將符合的ETF代碼轉換為List
CrudeOilETFList = newCrudeOilETF['Symbol'].values.tolist()
GoldETList = newGoldETF['Symbol'].values.tolist() 

SymbolList = CrudeOilETFList+GoldETList
print(CrudeOilETFList)
print(GoldETList)

['USO', 'UCO', 'DBO', 'SCO', 'USL', 'UWTI', 'DTO', 'DWTI', 'OLO', 'OLEM', 'SZO']
['GLD', 'IAU', 'SGOL', 'OUNZ', 'UGLD', 'DGL', 'UGL', 'DGP', 'GLL', 'DZZ', 'DGLD', 'DGZ', 'UBG']


In [40]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/USO?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=Tgw6xSDlmOe&corsDomain=finance.yahoo.com"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [57]:
etfs=SymbolList
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    for Symbol in SymbolList:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data

In [58]:
all_data_week = get_Date('1wk','adjclose')
all_data_month = get_Date('1mo','adjclose')

----------1wk----------
start crawling : USO
start crawling : UCO
start crawling : DBO
start crawling : SCO
start crawling : USL
start crawling : UWTI
已下市
start crawling : DTO
start crawling : DWTI
已下市
start crawling : OLO
start crawling : OLEM
start crawling : SZO
start crawling : GLD
start crawling : IAU
start crawling : SGOL
start crawling : OUNZ
start crawling : UGLD
start crawling : DGL
start crawling : UGL
start crawling : DGP
start crawling : GLL
start crawling : DZZ
start crawling : DGLD
start crawling : DGZ
start crawling : UBG
----------1mo----------
start crawling : USO
start crawling : UCO
start crawling : DBO
start crawling : SCO
start crawling : USL
start crawling : UWTI
已下市
start crawling : DTO
start crawling : DWTI
已下市
start crawling : OLO
start crawling : OLEM
start crawling : SZO
start crawling : GLD
start crawling : IAU
start crawling : SGOL
start crawling : OUNZ
start crawling : UGLD
start crawling : DGL
start crawling : UGL
start crawling : DGP
start crawling : GLL

In [59]:
all_data_week

,USO,UCO,DBO,SCO,USL,DTO,OLO,OLEM,SZO,GLD,...,OUNZ,UGLD,DGL,UGL,DGP,GLL,DZZ,DGLD,DGZ,UBG
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-28,11.00,25.080000,8.907330,66.820000,17.020000,158.119995,4.630,14.900000,91.699997,101.459999,...,10.520,73.699997,34.113441,29.730000,18.139999,115.830002,8.87,94.839996,17.100000,27.150000
2016-01-04,9.80,19.959999,7.962464,82.519997,15.650000,195.479996,6.410,13.550000,103.230003,105.680000,...,10.960,83.099998,35.451992,32.279999,19.670000,106.330002,8.14,83.690002,16.389999,28.610001
2016-01-11,8.79,15.980000,7.165233,100.419998,14.040000,223.789993,5.550,12.000000,112.320000,104.080002,...,10.820,79.300003,34.959877,31.270000,19.080000,109.510002,8.42,86.849998,16.629999,28.010000
2016-01-18,9.27,17.540001,7.558927,86.970001,14.900000,209.960007,5.350,12.800000,109.050003,105.000000,...,10.910,81.199997,35.333889,31.780001,19.440001,107.230003,8.24,84.330002,16.490000,28.240000
2016-01-25,9.65,18.920000,7.873882,78.400002,15.880000,198.440002,5.370,13.700000,105.349998,106.949997,...,11.100,85.800003,35.904739,33.000000,20.090000,103.650002,7.93,79.870003,16.190001,28.610001
2016-02-01,8.91,15.900000,7.352237,88.339996,15.250000,227.119995,5.400,13.060000,112.849998,112.320000,...,11.669,99.400002,37.764935,36.330002,22.150000,93.290001,7.11,68.629997,15.350000,28.610001
2016-02-08,8.33,13.840000,7.086494,94.750000,14.570000,246.740005,5.600,12.470000,114.970001,118.360001,...,12.310,115.599998,39.782608,40.259998,24.420000,83.349998,6.21,57.369999,14.420000,32.220001
2016-02-15,8.55,14.280000,7.047124,89.790001,14.650000,238.410004,5.750,12.530000,115.000000,117.580002,...,12.210,113.000000,39.546394,39.770000,24.889999,84.339996,6.34,58.380001,14.550000,31.719999
2016-02-22,8.78,15.020000,7.125863,83.345001,15.000000,226.960007,5.830,13.000000,111.800003,117.110001,...,12.160,110.699997,39.329861,39.180000,24.250000,84.809998,6.40,58.730000,14.620000,31.500000


In [60]:
all_data_month

,USO,UCO,DBO,SCO,USL,DTO,OLO,OLEM,SZO,GLD,...,OUNZ,UGLD,DGL,UGL,DGP,GLL,DZZ,DGLD,DGZ,UBG
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01,9.65,18.920000,7.873882,78.400002,15.880000,198.440002,5.370,13.700000,105.349998,106.949997,...,11.100,85.800003,35.904739,33.000000,20.090000,103.650002,7.930,79.870003,16.190001,28.610001
2016-02-01,9.00,15.820000,7.263657,79.084999,15.360000,216.809998,5.850,13.300000,109.110001,118.639999,...,12.321,115.500000,39.871189,40.419998,24.559999,82.419998,6.200,56.389999,14.400000,31.500000
2016-03-01,9.70,17.820000,7.647508,65.470001,16.309999,184.800003,5.550,14.290000,100.279999,117.639999,...,12.220,111.800003,39.644817,39.689999,25.030001,83.029999,6.570,56.720001,14.480000,31.889999
2016-04-01,11.30,23.580000,8.464424,45.830002,18.340000,128.289993,5.940,16.440001,84.129997,123.650002,...,12.840,128.899994,41.632961,43.459999,28.040001,74.339996,5.640,48.290001,13.750000,32.080002
2016-05-01,11.87,25.780001,8.917172,40.860001,19.260000,125.220001,4.690,17.500000,80.099998,116.059998,...,12.080,105.800003,39.093647,38.110001,24.200001,83.750000,6.350,57.900002,14.600000,31.230000
2016-06-01,11.57,24.440001,9.015595,40.810001,19.350000,117.889999,4.690,17.100000,81.000000,126.470001,...,13.126,135.500000,42.489243,45.029999,28.530001,70.000000,5.210,43.740002,13.300000,34.330002
2016-07-01,9.76,17.160000,7.932937,55.105000,16.910000,154.020004,4.100,14.410000,99.820000,128.979996,...,13.420,142.500000,43.424263,46.740002,29.110001,66.660004,5.040,41.040001,13.100000,34.099998
2016-08-01,10.36,18.940001,8.297104,47.419998,17.709999,140.470001,4.250,15.690000,89.139999,124.779999,...,12.960,12.760000,41.603436,43.639999,26.950001,71.279999,5.400,45.380001,13.670000,34.330002


週報酬率=(本週收盤價-上週收盤價)/上週收盤價  
月報酬率=(本月收盤價-上月收盤價)/上月收盤價

In [61]:
#測試用資料
data = [0.0089,0.0012,-0.002,0.01,-0.0002,0.02,0.03,0.01,-0.003,0.01,0.0102,-0.01]

In [50]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASKSR(R, rf):
    x = np.asarray(R)
    u = mean(x)/x.size    #期望值
    v = var(x, ddof=1)    #變異數
    SD = math.sqrt(v)     #標準差
    S = skew(x)           #偏度
    K = kurtosis(x)       #峰度

    c1 = 3*K-4*pow(S,2)-9
    c2 = 3*K-5*pow(S,2)-9
    a = ( 3*(cmath.sqrt(c1) ) / ( v*c2 ) )
    b = ( 3*S ) / ( SD*c2 )
    n = u - ( (3*S*SD) / c1 )
    d = 3*SD * ( (cmath.sqrt(c2)) / (c1) ) 
    phi = cmath.sqrt(pow(a,2)*pow(b,2))
    a_ = b + ( a*(n-rf) / cmath.sqrt(pow(d,2)+pow(n-rf,2))) 
    ASKSR = cmath.sqrt(2*(a_*(n-rf)-d*(phi-cmath.sqrt(pow(a,2)-pow(b-a_,2)))))
    
    return ASKSR
    

In [51]:
sharpeASKSR(data,0)

44.82677858843439j

In [52]:
def OmegaSharpeRatio(R, L):
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [53]:
OmegaSharpeRatio(data, 0)

5.598684210526315

In [ ]:
def Qfunction():
    